<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [1]:
from google.colab import drive
drive.mount('/content/drive') # , force_remount=True

address = "University/Term 6/MIR/Homeworks/HW3"
import sys
sys.path.append(f"/content/drive/My Drive/{address}")

%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


In [2]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
!pip install hazm
!pip install unidecode
# !pip install pandas
# !pip install nltk
# !pip install torch
# !pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 4.9 MB/s 
     |████████████████████████████████| 1.4 MB 34.1 MB/s 
     |████████████████████████████████| 233 kB 68.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=967bbb04f8fe158af0f0411601c98ea389c63b0df2121c18006b4d46c07372fc
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154511 sha256=90d928a16eaabac80d390d0af1bf4473782c6d13f52e5122ab3bb9f79ba88053
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Looking in indexes: ht

In [3]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import faiss
import pickle
import zipfile
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from transformers import pipeline
from IPython.display import display
from sentence_transformers import SentenceTransformer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [4]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [ ]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            category_PER = soup.find_all('li', {'class': 'breadcrumb-item'})
            category_PER = list(map(lambda x: x.text.strip(), category_PER))[1:]
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category-PER': category_PER,
                'category-ENG': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [ ]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [5]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [7]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. در ادامه، به توضیح هر یک از مراحل پیش‌پردازش می‌پردازیم.
    </p>
</div>

In [8]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]


def save_preprocessed_texts(texts, path="preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

In [10]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


preprocessor = Preprocessor(stopwords_path='stopwords.txt')

# texts = [data_to_text(data) for _, data in dataset.iterrows()]
# preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
# save_preprocessed_texts(preprocessed_texts)

preprocessed_texts = load_preprocessed_texts()

In [12]:
dataset

,date,title,intro,body,category
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,Politics
...,...,...,...,...,...
68357,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۵۴,از زینک چه می‌دانید؟ | علائم کمبود زینک یا روی,بدن شما نیاز به انواع مواد مغذی دارد تا عملکرد...,به گزارش همشهری آنلاین، بدن شما نیاز به انواع ...,LifeSkills
68358,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۰۲,یک راهکار ساده برای مطمئن شدن از کیفیت عینک آف...,استفاده از عینک آفتابی یک کار ضروری برای محافظ...,به گزارش همشهری آنلاین، بررسی‌ها نشان داده قرا...,LifeSkills
68359,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۸:۰۰,تصاویری از سرزمین لک‌لک‌های ایران | زریوار میز...,بهار فصل خوبی برای سفر به کردستان است؛ جاده‌ها...,به گزارش همشهری آنلاین، شنیدن صدای لک‌لک‌ها یک...,LifeSkills
68360,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۷:۵۵,۱۰ ترفند خانگی که سردردتان را بدون نیاز به قرص...,سردرد مشکل نادری نیست و همه‌ ما گاهی دچارش می‌...,به گزارش همشهری آنلاین به نقل از سلامت نیوز، ت...,LifeSkills


<div>
    <h3 style='direction:rtl;text-align:justify;'>
         ترنسفرمر
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش یک کلاس تعریف می‌کنیم که بتواند کارهایی که نیاز داریم را با استفاده از مدلی بر پایه‌ی ترنسفرمرها انجام دهد. برای این کار از SentenceTransformer در کتاب‌خانه‌ی HuggingFace بهره می‌بریم و مدل ParsBERT که به دست هوش‌واره فراهم شده است را به عنوان مدل زبانی آن استفاده می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در آغاز که یک شی با استفاده از این کلاس ساخته می‌شود یک SentenceTransformer با استفاده از مدل ParsBERT می‌سازیم تا از آن برای به دست آوردن embeddingها استفاده کنیم. سپس در صورتی که GPU در دسترس باشد این مدل را به روی آن منتقل می‌کنیم. علاوه بر این می‌توانیم یک پیش‌پردازشگر هم به متد سازنده بدهیم. این همان پیش‌پردازشگری است که دیتاست اولیه با آن پردازش شده است. ابزار ما از آن برای پیش‌پردازش درخواست‌ها در زمان تشخیص و تبدیل آن‌ها به فرمتی مطابق با متن اولیه پس از پیش‌پردازش استفاده خواهد کرد.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در تابع train_embeddings لیستی از متن‌ها یا لیستی از متن‌های شکسته شده به کلمات را دریافت می‌کنیم تا امبدینگ‌های مربوط به آن‌ها را به دست بیاوریم. در حالت دوم، پیش از ادامه‌ی کار کلمات را در یک متن واحد در کنار هم قرار می دهیم. توجه می‌کنیم که داده‌ها پس از پیش‌پردازش در حالت دوم قرار دارند. در نهایت پس از به دست آمدن امبدینگ‌ها، آن‌ها را در یک آرایه‌ی نامپای ذخیره می‌کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از به دست آمدن امبدینگ‌ها می‌توانیم آن‌ها را با استفاده از متد save_embeddings در آدرس دلخواهمان به عنوان یک فایل ذخیره کنیم و در دفعات بعدی به جای محاسبه‌ی دوباره، آن‌ها را با استفاده از متد load_embeddings بازیابی کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        وقتی که امبدینگ‌ها را محاسبه یا بازیابی کردیم، می‌توانیم با استفاده از متد make_index نمایه‌هایی را با استفاده از ابزار Faiss ایجاد کنیم. این ابزار در به دست آوردن موارد مشابه به درخواست‌ها به ما کمک خواهد کرد.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        در نهایت پس از اینکه تمام مراحل گفته شده را طی کردیم می‌توانیم با استفاده از متد predict یک درخواست را به ابزارمان بدهیم و این ابزار پس از به دست آوردن امبدینگ مربوط به آن درخواست، تعداد دلخواهی از موارد مربوط به آن را با توجه به مدل ترنسفرمر استفاده شده و نمایه‌ی Faiss تولید شده به ما خواهد داد.
    </p>
</div>


In [11]:
class Transformer:

    def __init__(self, preprocessor=None):
        self.model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
        self.preprocessor = preprocessor
        if torch.cuda.is_available():
            self.model = self.model.to(torch.device("cuda"))
        self.embeddings = None
        self.index = None
    
    def train_embeddings(self, train_dataset: list):
        if type(train_dataset[0]) == list:
            train_dataset = list(map(lambda doc: ' '.join(doc), train_dataset))
        self.embeddings = self.model.encode(train_dataset, show_progress_bar=True)
        self.embeddings = np.array(
            [embedding for embedding in self.embeddings]).astype("float32")
    
    def make_index(self, dataset: list):
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index = faiss.IndexIDMap(self.index)
        self.index.add_with_ids(self.embeddings, np.array(list(range(len(dataset)))))

    def save_embeddings(self, path):
        with open(path, "wb") as f:
            pickle.dump(self.embeddings, f)
        
    def load_embeddings(self, path):
        with open(path, "rb") as f:
            self.embeddings = pickle.load(f)
    
    def predict(self, query: str, dataset: pd.DataFrame, k=10):
        if self.preprocessor:
            query = ' '.join(self.preprocessor.preprocess(query))
        vector = self.model.encode(list([query]))
        D, I = self.index.search(np.array(vector).astype("float32"), k=k)
        return dataset.iloc[I.flatten().tolist()]
        

In [33]:
transformer = Transformer(preprocessor)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [34]:
transformer.train_embeddings(dataset.title.to_list())

Batches:   0%|          | 0/2137 [00:00<?, ?it/s]

In [35]:
transformer.save_embeddings("Transformer_model.pickle")

In [36]:
transformer.load_embeddings("Transformer_model.pickle")

In [37]:
transformer.make_index(dataset.title.to_list())

In [38]:
transformer.predict("فدراسیون فوتبال", dataset)

,date,title,intro,body,category
28924,جمعه ۱۷ اردیبهشت ۱۴۰۰ - ۱۰:۴۴,تشکیل تیم فوتبال بهبودیافتگان,اداره ورزش شهرداری منطقه ۱۶ قصد دارد از میان ب...,مدیر اداره ورزش شهرداری منطقه ۱۶ با بیان اینکه...,Society
66713,پنجشنبه ۲۳ دی ۱۴۰۰ - ۰۱:۳۲,عکس روز| فوتبال قطع عضو,بازیکنان فوتبال قطع عضو در یک مسابقه مقدماتی د...,فوتبال قطع عضو گونه‌ای از فوتبال است که برای ا...,LifeSkills
51404,پنجشنبه ۷ بهمن ۱۴۰۰ - ۱۷:۵۳,ویدیو | صحبت های رئیس فدراسیون فوتبال در ورزشگ...,رئیس فدراسیون فوتبال در بدو ورود به ورزشگاه آز...,به گزارش همشهری آنلاین، شهاب الدین عزیزی خادم ...,Sport
48243,پنجشنبه ۱۸ شهریور ۱۴۰۰ - ۱۶:۲۶,قول رئیس فیفا به هواداران فوتبال ایران و عزیزی...,رئیس فدراسیون جهانی فوتبال برای هواداران در ای...,به گزارش همشهری‌آنلاین، جیوانی اینفانتینو، رئی...,Sport
48980,سه‌شنبه ۶ مهر ۱۴۰۰ - ۱۰:۳۲,محکومیت باشگاه استقلال در کمیته تعیین وضعیت فد...,باشگاه استقلال در دو پرونده از سوی کمیته تعیین...,به گزارش همشهری‌آنلاین به نقل از سایت فدراسیون...,Sport
46967,شنبه ۱۹ تیر ۱۴۰۰ - ۱۲:۲۴,عکس | تجمع هواداران ذوب‌آهن مقابل فدراسیون فوتبال,تعدادی از هواداران تیم فوتبال ذوب‌آهن اصفهان م...,به گزارش همشهری‌آنلاین به نقل از فارس، جمعی از...,Sport
49684,چهارشنبه ۲۴ آذر ۱۴۰۰ - ۱۵:۲۷,آشنایی با سازمان لیگ فوتبال ایران,سازمان لیگ فوتبال ایران در ۱۱ آبان ۱۳۸۰ شکل گر...,به گزارش همشهری آنلاین، سازمان لیگ فوتبال ایر...,Sport
50902,پنجشنبه ۲۸ بهمن ۱۴۰۰ - ۱۷:۳۶,سرپرست فدراسیون فوتبال نیامده تغییر کرد,در حالی که صبح امروز سرپرست فدراسیون فوتبال تع...,به گزارش همشهری آنلاین پس از عزل موقت شهاب‌الد...,Sport
45829,شنبه ۱۸ اردیبهشت ۱۴۰۰ - ۱۲:۰۴,عکس | نماز جماعت رقیب استقلال در زمین فوتبال,اقدام عبادی مربی و بازیکنان الهلال پیش از دیدا...,به گزارش همشهری‌آنلاین، تیم فوتبال الهلال عربس...,Sport
49259,سه‌شنبه ۱۸ آبان ۱۴۰۰ - ۱۹:۰۵,فدراسیون فوتبال ایران در خطر تعلیق,در صورتی که اساسنامه فدراسیون تا اواسط اسفندما...,به گزارش همشهری آنلاین به نقل از ورزش سه، بحث ...,Sport


In [39]:
title_transformer = transformer

In [12]:
transformer = Transformer(preprocessor=preprocessor)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [41]:
# transformer.train_embeddings(preprocessed_texts)

Batches:   0%|          | 0/2137 [00:00<?, ?it/s]

In [42]:
# transformer.save_embeddings("processed_embeddings")

In [44]:
transformer.load_embeddings("processed_embeddings")

In [45]:
transformer.make_index(preprocessed_texts)

In [16]:
transformer.predict("پیروزی ایران در مقابل امارات", dataset, 10)

,date,title,intro,body,category
46254,سه‌شنبه ۲۵ خرداد ۱۴۰۰ - ۲۱:۵۶,پایان نیمه اول ایران-عراق | برتری ایران با گل ...,سردار ازمون همانند بازی با بحرین برابر عراق هم...,به گزارش همشهری آنلاین تیم دیدار تیم‌های ملی ...,Sport
29991,سه‌شنبه ۱۸ خرداد ۱۴۰۰ - ۰۰:۱۶,تصاویر | ۶ قاب از برد شیرین ایران در برابر بحرین,تیم ملی فوتبال امشب در دومین بازی دور برگشت مر...,,Society
49229,پنجشنبه ۲۰ آبان ۱۴۰۰ - ۱۶:۳۸,پیروزی کره برابر امارات | چشم بادامی ها در تعق...,شاگردان بنتو با پیروزی برابر امارات ۱۱ امتیازی...,به گزارش همشهری آنلاین دیدار دو تیم کره جنوبی ...,Sport
48840,جمعه ۱۶ مهر ۱۴۰۰ - ۰۰:۲۵,عذرخواهی سردار آزمون از مردم ایران,مهاجم تیم ملی فوتبال کشورمان با انتشار متنی در...,به گزارش همشهری آنلاین به نقل از ایرنا، تیم مل...,Sport
48835,جمعه ۱۶ مهر ۱۴۰۰ - ۰۸:۲۶,واکنش ستاره سرشناس سابق هلند به پیروزی ایران م...,کلارنس سیدورف، ستاره سابق تیم ملی هلند، میلان ...,به گزارش همشهری‌آنلاین، دیدار تیم‌های ملی فوتب...,Sport
51268,سه‌شنبه ۱۲ بهمن ۱۴۰۰ - ۲۰:۰۵,ویدیو | گل ایران به امارات | درخشش ادامه دار ط...,تیم ملی فوتبال ایران موفق شد یک برصفر امارات ر...,به گزارش همشهری آنلاین، مهدی طارمی که در بازی ...,Sport
47894,دوشنبه ۴ مرداد ۱۴۰۰ - ۱۸:۰۸,زمان دیدار پرسپولیس و استقلال در لیگ قهرمانان ...,پرسپولیس در مرحله یک هشتم باید به مصاف رقیب تا...,به گزارش همشهری آنلاین دیدار تیم‌های فوتبال پر...,Sport
48844,پنجشنبه ۱۵ مهر ۱۴۰۰ - ۲۱:۵۰,اتفاق کم‌سابقه در بازی ایران و امارات | شجاع خ...,در دقیقه ۶۰ بازی ایران و امارات در حالیکه شجاع...,به گزارش همشهری آنلاین پس از اخراج شجاع خلیل ز...,Sport
50570,دوشنبه ۱۳ دی ۱۴۰۰ - ۱۸:۰۲,ویدیو | ۲ گل نیمه اول بازی تراکتور و پرسپولیس,تیم های پرسپولیس و تراکتور در هفته سیزدهم لیگ ...,به گزارش همشهری آنلاین، در این دیدار عیسی آل ک...,Sport
51031,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۲:۰۷,ببینید | فرار الاهلی از شکست با درخشش ۲ ستاره ...,۲ بازیکن ایرانی الاهلی را از شکست نجات دادند.,به گزارش همشهری آنلاین در ادامه هفته پانزدهم ل...,Sport


In [17]:
transformer.predict("حمله‌ی روسیه به اکراین", dataset)

,date,title,intro,body,category
17768,پنجشنبه ۵ اسفند ۱۴۰۰ - ۲۳:۲۷,تلخ‌ترین تصویر از فرار مردم اوکراین پس از حمله...,از صبح امروز ارتش روسیه به اوکراین حمله کرده و...,,World
17074,دوشنبه ۱۸ بهمن ۱۴۰۰ - ۰۲:۰۴,ببینید | خط و نشان اتمی روسیه برای اوکراین با ...,روسیه ۲ بمب افکن خود را در نزدیکی اوکراین به پ...,به گزارش همشهری آنلاین به قل از باشگاه خبرنگار...,World
17786,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۷:۲۷,تصاویر | آتش سوزی مشکوک مقابل وزارت دفاع اوکراین,رسانه‌ها با انتشار تصاویری از مقابل ساختمان وز...,به گزارش همشهری آنلاین، حملات ارتش روسیه تا کن...,World
17750,جمعه ۶ اسفند ۱۴۰۰ - ۰۹:۵۷,عکس | چهره خاص بایدن پس از آغاز حمله روسیه به ...,بایدن در سخنانی تحریم‌های جدید علیه روسیه را ا...,منبع: مشرق\n\n\nلحظه به لحظه با آخرین گزارش‌ها...,World
63461,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
17859,یکشنبه ۱ اسفند ۱۴۰۰ - ۱۹:۲۳,بشنوید | مهم‌ترین اخبار امروز از بحران اوکراین,لفاظی‌ها بین مقامات روسیه و طرف‌های غربی بر سر...,به گزارش همشهری آنلاین، درحالی که سران غرب از ...,World
16999,پنجشنبه ۲۸ بهمن ۱۴۰۰ - ۲۰:۴۶,ببینید | پای بمب اتم به جنگ لفظی شرق و غرب باز...,در اوج درگیری‌ها بر سر اوکراین، روسیه و بلاروس...,به گزارش همشهری آنلاین به نقل از فارس، رئیس‌جم...,World
17628,یکشنبه ۸ اسفند ۱۴۰۰ - ۲۱:۵۴,محل برخورد اسکندرهای پرتاب‌ شده از بلاروس مشخص شد,چندی پیش یکی از مشاوران وزیر کشور اوکراین گزار...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World
12363,شنبه ۱۶ بهمن ۱۴۰۰ - ۱۳:۱۸,جدیدترین تصاویر از اسکورت رئیسی در سفر به روسیه,تیم اسکورت رئیس‌جمهور «ابراهیم رئیسی» در مسیر ...,,Politics
17339,شنبه ۲۱ اسفند ۱۴۰۰ - ۱۵:۴۶,جدیدترین نقشه‌ای که اوضاع کی‌یف را نشان می‌دهد...,از صبح امروز بیشتر مسائل مربوط به کی‌یف، پایتخ...,به گزارش همشهری آنلاین و به نقل از شبکه خبری ب...,World


In [46]:
transformer.predict("صعود ایران به جام جهانی", dataset)

,date,title,intro,body,category
51395,پنجشنبه ۷ بهمن ۱۴۰۰ - ۲۰:۵۳,تصاویر اختصاصی از جشن صعود تیم ملی به جام جهانی,تیم ملی فوتبال کشورمان با غلبه بر عراق به جام ...,,Sport
49139,سه‌شنبه ۲۵ آبان ۱۴۰۰ - ۲۱:۵۵,جایگاه ایران در جدول گروه A مقدماتی جام‌ جهانی,ایران با برد ۳ بر صفر مقابل سوریه با ۱۶ امتیا...,,Sport
47744,شنبه ۹ مرداد ۱۴۰۰ - ۲۱:۲۳,جدول | المپیک از فردا برای ایران شروع می‌شود,با آغاز رقابت‌های کشتی در المپیک امید ایران از...,,Sport
46142,دوشنبه ۳۱ خرداد ۱۴۰۰ - ۱۳:۳۲,عکس | واکنش فیفا به گل خاطره‌انگیز حمید استیلی,فیفا در صفحه توییتر «جام جهانی» به اولین پیروز...,به گزارش همشهری‌آنلاین، ۲۳ سال پیش در چنین روز...,Sport
29991,سه‌شنبه ۱۸ خرداد ۱۴۰۰ - ۰۰:۱۶,تصاویر | ۶ قاب از برد شیرین ایران در برابر بحرین,تیم ملی فوتبال امشب در دومین بازی دور برگشت مر...,,Society
12647,جمعه ۸ بهمن ۱۴۰۰ - ۲۰:۱۲,واکنش آمریکا به صعود تیم ملی ایران به جام جهان...,نماینده ویژه آمریکا در امور ایران صعود ایران ب...,به گزارش همشهری آنلاین، رابرت مالی در توئیتر ن...,Politics
46610,چهارشنبه ۵ خرداد ۱۴۰۰ - ۱۲:۱۸,عکس | ایران - استرالیا جزو ۵ بازی خاطره‌انگیز ...,کنفدراسیون فوتبال آسیا دیدار برگشت ایران با اس...,به گزارش همشهری‌آنلاین، سایت کنفدراسیون فوتبال...,Sport
49105,پنجشنبه ۲۷ آبان ۱۴۰۰ - ۱۲:۲۳,ویدیو | گل ملی پوش ایران در میان برترین گل های...,کنفدراسیون فوتبال آسیا گل های برتر هفته ششم م...,به گزارش همشهری آنلاین، در این انتخاب گل سوم ا...,Sport
51893,پنجشنبه ۱۲ اسفند ۱۴۰۰ - ۲۱:۱۲,فیفا اعلام کرد؛ ورزشگاه دیدار ایران و لبنان مش...,سایت کنفدراسیون جهانی فوتبال فیفا ورزشگاه محل ...,به گزارش همشهری آنلاین، دیدار ایران و لبنان در...,Sport
51394,پنجشنبه ۷ بهمن ۱۴۰۰ - ۲۰:۵۴,عکس | واکنش جالب فیفا به صعود ایران به جام جها...,تیم ملی ایران به عنوان اولین تیم آسیایی موفق ب...,به گزارش تیم ملی فوتبال کشورمان موفق شد در دید...,Sport


In [22]:
transformer.predict("بهترین گوشی بازار", dataset, 10)

,date,title,intro,body,category
23813,جمعه ۲۶ آذر ۱۴۰۰ - ۰۹:۱۸,جدیدترین قیمت گوشی های ۳۰ میلیونی در بازار | ب...,این گزارش به بررسی انواع گوشی‌های ۲۸ تا ۳۰ میل...,به گزارش همشهری آنلاین به نقل از خبرآنلاین،در ...,Economy
26942,چهارشنبه ۲۱ اردیبهشت ۱۴۰۱ - ۲۰:۰۱,با ۱۰ میلیون چه موبایلی بخریم؟ | جدول جدیدترین...,آنچه در زیر می‌آید قیمت انواع موبایل ۷ تا ۱۰می...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
26035,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۴:۲۵,عکس | جدیدترین قیمت شاسی بلند سایپا,در گزارش زیر قیمت روز خودرو چانگان سایپا در با...,به گزارش همشهری آنلاین، در جدول زیر قیمت روز ا...,Economy
24630,یکشنبه ۱۲ دی ۱۴۰۰ - ۲۳:۵۷,قیمت جدید انواع ماهی در بازار تهران | جدول قیم...,گران شدن ماهی موجب شده است که بسیاری از خانواد...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
26575,پنجشنبه ۱۸ فروردین ۱۴۰۱ - ۱۱:۰۷,جدیدترین قیمت برنج در بازار اعلام شد,در این گزارش قیمت انواع برنج در بازار تهران بر...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
26572,پنجشنبه ۱۸ فروردین ۱۴۰۱ - ۱۱:۵۵,ببینید | عرضه گوشت تنظیم بازاری از امروز در بازار,گوشت منجمد گوساله و گوسفند از امروز با قیمت من...,به گزارش همشهری آنلاین به نقل از صدا و سیما، ب...,Economy
27025,دوشنبه ۱۹ اردیبهشت ۱۴۰۱ - ۰۹:۳۶,جدیدترین قیمت ماهی در بازار | قزل آلا کیلویی چند؟,در این گزارش قیمت جدید ماهی در سطح بازار تهران...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
67292,چهارشنبه ۱۳ بهمن ۱۴۰۰ - ۰۶:۵۰,اینفوگرافیک | خودروهای مورد علاقه مردم در سراس...,خودروهای مورد علاقه مردم در سراسر جهان را در ا...,منبع: تسنیم,LifeSkills
24801,یکشنبه ۵ دی ۱۴۰۰ - ۰۹:۳۴,قیمت روز انواع ماهی بسته‌بندی در بازار تهران |...,در این گزارش قیمت انواع ماهی در بازار تهران را...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
26273,چهارشنبه ۳۱ فروردین ۱۴۰۱ - ۰۸:۴۳,جدیدترین قیمت گوشت در بازار اعلام شد | گوشت گو...,در این گزارش قیمت روز گوشت قرمز در سطح بازار ت...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy


In [25]:
transformer.predict("ماهی شب عید", dataset, 10)

,date,title,intro,body,category
24801,یکشنبه ۵ دی ۱۴۰۰ - ۰۹:۳۴,قیمت روز انواع ماهی بسته‌بندی در بازار تهران |...,در این گزارش قیمت انواع ماهی در بازار تهران را...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
27833,چهارشنبه ۲۵ فروردین ۱۴۰۰ - ۲۱:۲۲,تصاویر | تهران در شب اول رمضان,تصاویری از تهران در نخستین افطار ماه رمضان را ...,,Society
25880,یکشنبه ۱۵ اسفند ۱۴۰۰ - ۱۲:۰۳,نرخ جدید انواع ماهی در بازار اعلام شد | ماهی ز...,مسئول آبزیان اتحادیه فروشندگان پرنده و ماهی با...,به گزارش همشهری آنلاین به نقل از ایلنا، فخرالد...,Economy
24630,یکشنبه ۱۲ دی ۱۴۰۰ - ۲۳:۵۷,قیمت جدید انواع ماهی در بازار تهران | جدول قیم...,گران شدن ماهی موجب شده است که بسیاری از خانواد...,به گزارش همشهری آنلاین به نقل از اقتصاد آنلاین...,Economy
15398,شنبه ۳ اردیبهشت ۱۴۰۱ - ۱۲:۵۳,تصاویر همشهری آنلاین از رئیسی در شب قدر؛ رئیس ...,مراسم احیای شب بیست و یکم ماه رمضان بامداد امر...,,Politics
65985,یکشنبه ۱۶ آبان ۱۴۰۰ - ۲۰:۰۹,عکس روز| «شب آتش» کرونایی,جشن‌های سنتی «شب آتش» در شهر «لویس» انگلیس با ...,Dan Kitwood/Getty Images,LifeSkills
26406,پنجشنبه ۲۵ فروردین ۱۴۰۱ - ۰۹:۴۱,ماهی ارزان شد | قزل‌آلا چند؟,مسئول حوزه آبزیان اتحادیه فروشندگان مرغ و ماهی...,به گزارش همشهری آنلاین به نقل از ایلنا، فخرالد...,Economy
66898,چهارشنبه ۸ دی ۱۴۰۰ - ۲۲:۵۶,عکس روز| درخت کریسمس برای فیل,یک فیل در باغ‌وحش برلین پایتخت آلمان یک درخت ک...,بسیاری از درختان تزئینی کریسمس در نهایت به عنو...,LifeSkills
68238,شنبه ۳ اردیبهشت ۱۴۰۱ - ۱۸:۵۴,تصاویر متفاوت حضور زنان در شب قدر ؛ اینجا امام...,مراسم احیای شب قدر بیست و یکم ماه رمضان ۱۴۰۱ د...,منبع» باشگاه خبرنگاران جوان,LifeSkills
68232,یکشنبه ۴ اردیبهشت ۱۴۰۱ - ۱۴:۱۳,عکس | واکنش بانمک کودک به قرآن بر سر گذاشتن ما...,رفتار جالب کودک به قرآن بر سر گذاشتن مادر را م...,,LifeSkills


In [ ]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts

mini_dataset, mini_preprocessed_texts = get_mini_dataset()

In [26]:
transformer = Transformer(preprocessor)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [28]:
transformer.train_embeddings(mini_preprocessed_texts)

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

In [29]:
transformer.save_embeddings("mini_embeddings")

In [30]:
transformer.load_embeddings("mini_embeddings")

In [31]:
transformer.make_index(mini_preprocessed_texts)

In [33]:
transformer.predict("بهترین گوشی بازار", mini_dataset, 10)

,date,title,intro,body,category
3353,شنبه ۳۱ اردیبهشت ۱۴۰۱ - ۱۰:۴۲,معرفی بهترین گوشی تا قیمت ۸ میلیون تومان,شما می توانید با بررسی معروف ترین و بهترین گوش...,از سوی دیگر در سال های اخیر قیمت مربوط به گوشی...,IT
3235,یکشنبه ۲۹ اسفند ۱۴۰۰ - ۱۲:۲۹,گوشی میان‌رده موتورولا چه مشخصاتی دارد؟,گزارش‌های اخیر نشان می‌دهد که موتورولا در حال ...,به گزارش همشهری آنلاین و به نقل از گیزموچاینا،...,IT
3498,جمعه ۱ بهمن ۱۴۰۰ - ۱۲:۵۹,دسته بازی موبایلی لنوو همراه با گوشی فن‌دار Le...,لنوو به تازگی وسیله جانبی جدیدی را برای بازی ب...,به گزارش همشهری آنلاین و به نقل از گیزموچاینا،...,IT
3401,شنبه ۲۰ فروردین ۱۴۰۱ - ۱۴:۴۴,سازنده عجله‌ای ندارد | تاشوی وان‌پلاس؛ شاید وق...,به نظر می‌رسد که در سال جاری میلادی شاهد ورود ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3233,چهارشنبه ۶ بهمن ۱۴۰۰ - ۱۱:۱۹,هشدار جدی برای اپل و سامسونگ | حمله گوشی‌ساز چ...,Realme چین ماه آینده با گران‌ترین دستگاه خود و...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,IT
3520,چهارشنبه ۱۵ دی ۱۴۰۰ - ۱۸:۳۷,ردمی K50 شیائومی؛ ماه آینده به بازار می‌آید | ...,شرکت شیائومی امروز تایید کرد که گوشی «ردمی کی...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
2026,پنجشنبه ۷ بهمن ۱۴۰۰ - ۱۷:۵۳,ویدیو | صحبت های رئیس فدراسیون فوتبال در ورزشگ...,رئیس فدراسیون فوتبال در بدو ورود به ورزشگاه آز...,به گزارش همشهری آنلاین، شهاب الدین عزیزی خادم ...,Sport
3221,چهارشنبه ۲۹ دی ۱۴۰۰ - ۰۹:۵۹,مشخصات نخستین تبلت ویوو فاش شد,پس از شایعات فراوان، معاون اجرایی ویوو (Vivo) ...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT
3313,سه‌شنبه ۷ دی ۱۴۰۰ - ۱۹:۱۹,داغ شدن رقابت تاشوها | گوشی جدید شیائومی به با...,شیائومی که گوشی تاشوی Mi Mix Fold را در کارنام...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3467,جمعه ۱ بهمن ۱۴۰۰ - ۱۰:۵۷,نسخه ویژه پرچم‌دار موتورولا به‌زودی عرضه خواهد شد,ماه گذشته، موتورولا متعلق به لنوو، جدیدترین گو...,به گزارش همشهری آنلاین و به نقل از وب‌سایت گیز...,IT


In [35]:
transformer.predict("پیروزی ایران در مقابل امارات", mini_dataset, 10)

,date,title,intro,body,category
2086,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۲:۰۷,ببینید | فرار الاهلی از شکست با درخشش ۲ ستاره ...,۲ بازیکن ایرانی الاهلی را از شکست نجات دادند.,به گزارش همشهری آنلاین در ادامه هفته پانزدهم ل...,Sport
672,شنبه ۲۴ اردیبهشت ۱۴۰۱ - ۱۵:۲۸,رئیس جدید امارات انتخاب شد,ولی‌عهد ابوظبی رسماً به‌عنوان رئیس دولت امارات...,به گزارش همشهری آنلاین به نقل از فارس، پس از د...,World
2177,دوشنبه ۲۲ آذر ۱۴۰۰ - ۱۸:۵۰,تغییر ساعت دو دیدار پرسپولیس در هفته دهم و دوا...,ساعت دو مسابقه لیگ برتر از هفته های دهم و دواز...,به گزارش همشهری آنلاین، مسابقه دو تیم پیکان و ...,Sport
732,دوشنبه ۲۹ فروردین ۱۴۰۱ - ۰۲:۲۴,خیانتی دیگر؛ مشارکت امارات در سالروز تأسیس رژی...,امارات به عنوان یک کشور سازشگر در سالروز تأسیس...,به گزارش همشهری آنلاین، ایسنا به نقل از المیاد...,World
369,دوشنبه ۷ تیر ۱۴۰۰ - ۱۶:۴۳,توییت طعنه آمیز حسام آشنا به حامیان طالبان در...,مشاور پیشین ریاست جمهوری در توییتی به حمایت بر...,به گزارش همشهری‌آنلاین، حسام الدین آشنا مشاور ...,Politics
2192,سه‌شنبه ۱۶ شهریور ۱۴۰۰ - ۱۴:۵۸,داور دیدار استقلال - پرسپولیس مشخص شد,داور دیدار پرسپولیس مقابل استقلال تاجیکستان مش...,به گزارش همشهری‌آنلاین، تیم فوتبال پرسپولیس سه...,Sport
2080,پنجشنبه ۱۵ اردیبهشت ۱۴۰۱ - ۱۰:۵۸,حسرت بزرگ برای قائدی و نوراللهی؛ شکست تلخ و از...,الاهلی امارات شکست تلخی را برابر العین تجربه ک...,به گزارش همشهری آنلاین در بازی فینال کاپ امارا...,Sport
55,دوشنبه ۱۵ فروردین ۱۴۰۱ - ۱۸:۳۰,توئیت امیر عبداللهیان درباره مکث پیش آمده در م...,وزیر امور خارجه جمهوری اسلامی ایران با انتشار ...,به گزارش همشهری آنلاین، حسین امیر عبداللهیان ب...,Politics
2292,شنبه ۲۱ اسفند ۱۴۰۰ - ۱۶:۲۵,ساعت برگزاری دربی برگشت تغییر کرد,ساعت دیدار تیم های استقلال و پرسپولیس از هفته ...,به گزارش همشهری آنلاین به نقل از روابط عمومی س...,Sport
2291,پنجشنبه ۱۴ مرداد ۱۴۰۰ - ۰۸:۰۶,مصاف ایران و آمریکا بر سر ۲ مدال المپیک در کشت...,با رفتن رقیب آمریکایی اطری به دیدار رده بندی ا...,به گزارش همشهری آنلاین در جریان رقابت‌های کشتی...,Sport


In [36]:
transformer.predict("حمله‌ی روسیه به اکراین", mini_dataset)

,date,title,intro,body,category
3271,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
3550,شنبه ۷ اسفند ۱۴۰۰ - ۱۴:۴۴,اختلال وسیع در اینترنت اوکراین | سیگنال ضعیف است,پوشش اینترنت در اوکراین تحت تأثیر تهاجم روسیه ...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,IT
768,جمعه ۶ اسفند ۱۴۰۰ - ۰۲:۴۷,روسیه به دنبال این هدف در اوکراین است | تعداد ...,یک مقام دفاعی ایالات متحده می‌گوید، کی‌یف یکی ...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,World
764,جمعه ۶ اسفند ۱۴۰۰ - ۱۲:۱۹,ادعای وزیر دفاع انگلیس درباره نقشه روسیه برای ...,وزیر دفاع انگلیس در بیانیه‌ای مدعی شد، روسیه ق...,به گزارش همشهری آنلاین و به نقل از گاردین، بن ...,World
476,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۷:۵۳,فقط پیروزی! | ادعای جدید وزیر دفاع اوکراین درب...,اولکسی رزنیکوف، وزیر دفاع اوکراین روز سه‌شنبه ...,به گزارش همشهری آنلاین و به نقل از شبکه خبری س...,World
760,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۳:۱۱,تازه‌ترین درخواست وزیر امور خارجه اوکراین علیه...,دیمیترو کولبا، وزیر امور خارجه اوکراین، پس از ...,به گزارش همشهری آنلاین و به نقل از گاردین، او ...,World
417,جمعه ۶ اسفند ۱۴۰۰ - ۱۴:۴۸,عکس | رئیس جمهور اوکراین در خط مقدم جنگ,عکسی از زلنسکی رئیس جمهور اوکراین در مناطق جنگ...,به گزارش همشهری آنلاین، گفته می شود زلنسکی که ...,World
681,شنبه ۷ اسفند ۱۴۰۰ - ۲۲:۱۸,رئیس‌جمهور چچن: پوتین برای حمله به اوکراین تصم...,رئیس جمهوری چچن با بیان اینکه پوتین تصمیم درست...,به گزارش همشهری آنلاین به نقل از ایسنا، «رمضان...,World
3547,جمعه ۶ اسفند ۱۴۰۰ - ۱۹:۲۱,نبرد سایبری ادامه دارد | حمله سنگین هکرهای بلا...,مقامات اوکراینی روز جمعه ادعا کردند، هکرهایی ک...,به گزارش همشهری آنلاین و به نقل از شبکه خبری س...,IT
3222,دوشنبه ۲۳ اسفند ۱۴۰۰ - ۱۵:۳۹,روسیه سرانجام اینستاگرام را فیلتر کرد,بر اساس گزارش نت‌بلاکس شبکه اجتماعی اینستاگرام...,به گزارش همشهری آنلاین، درحالی که چند روز پیش ...,IT


In [37]:
transformer.predict("ماهی شب عید", mini_dataset, 10)

,date,title,intro,body,category
1365,پنجشنبه ۲۳ اردیبهشت ۱۴۰۰ - ۱۲:۴۵,شنبه تعطیل نیست,سخنگوی کمیسیون اجتماعی مجلس گفت: با توجه به مص...,به گزارش همشهری‌آنلاین به نقل از تسنیم، علی ب...,Society
2523,جمعه ۸ بهمن ۱۴۰۰ - ۱۷:۲۶,عکس روز| مارمولک ماهی و ته‌سیگار,یک مارمولک‌ماهی کوچک یک ته‌سیگار را با طعمه‌اش...,Photograph: Steven Kovacs,Science
3138,چهارشنبه ۳ فروردین ۱۴۰۱ - ۱۲:۴۶,⁠⁣تاریخ جشن نوروز,چرا اصطلاح «نوروز جمشیدی» از قدیم در ایران معم...,‌به گزارش همشهری آنلاین به نقل از ایسنا، در بخ...,Culture
2911,جمعه ۱۳ اسفند ۱۴۰۰ - ۱۸:۱۸,جزئیات تازه از برنامه نوروزی خندوانه از زبان ر...,ضبط ویژه برنامه نوروزی «خندوانه» از شنبه (۱۴ ا...,به گزارش همشهری‌ آنلاین به نقل از ایسنا، رامبد...,Culture
1561,پنجشنبه ۱۶ اردیبهشت ۱۴۰۰ - ۱۷:۵۵,تصمیم ستاد ملی کرونا برای تعطیلات عید فطر | مم...,علیرضا رییسی سخنگوی ستاد ملی مقابله با کرونا گ...,به گزارش همشهری آنلاین مشاغل گروه یک و ۲ در شه...,Society
2981,شنبه ۳ مهر ۱۴۰۰ - ۱۴:۴۴,برنامه تعطیلی سینماها در ایام اربعین,محمدرضا صابری، سخنگوی انجمن سینماداران گفت: سی...,به گزارش همشهری آنلاین به نقل از ایسنا، محمدرض...,Culture
158,سه‌شنبه ۲۹ تیر ۱۴۰۰ - ۲۲:۵۵,فتوای رهبر انقلاب درباره پخش قربانی میان نیازم...,رهبر معظم انقلاب درخصوص پخش قربانی میان مردم ن...,به گزارش همشهری‌آنلاین به نقل از میزان، حضرت آ...,Politics
2292,شنبه ۲۱ اسفند ۱۴۰۰ - ۱۶:۲۵,ساعت برگزاری دربی برگشت تغییر کرد,ساعت دیدار تیم های استقلال و پرسپولیس از هفته ...,به گزارش همشهری آنلاین به نقل از روابط عمومی س...,Sport
2826,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۰:۲۴,تصویر شهید حاج قاسم سلیمانی روی سیمرغ‌های جشنو...,اختتامیه چهلمین جشنواره فیلم فجر با پخش سرود و...,به گزارش همشهری آنلاین، سیمرغ های جشنواره چهلم...,Culture
3645,دوشنبه ۱ آذر ۱۴۰۰ - ۲۲:۱۸,عکس روز| بازی آوارگان,کودکان در اردوگاه آوارگان در شهر «تغز» یمن باز...,Anadolu Agency/Getty Images,LifeSkills


In [39]:
transformer.predict("احکام روزه", mini_dataset, 10)

,date,title,intro,body,category
3990,جمعه ۵ آذر ۱۴۰۰ - ۰۳:۱۳,عکس روز| سالمرگ کاسترو,دانشجویان در هاوانا پایتخت کوبا در پنجمین سالم...,Alexandre Meneghini/Reuters,LifeSkills
3645,دوشنبه ۱ آذر ۱۴۰۰ - ۲۲:۱۸,عکس روز| بازی آوارگان,کودکان در اردوگاه آوارگان در شهر «تغز» یمن باز...,Anadolu Agency/Getty Images,LifeSkills
3835,دوشنبه ۱۸ مرداد ۱۴۰۰ - ۰۱:۴۶,عکس روز | اعتراض با تیروکمان,یک تظاهرکننده در بانکوک پایتخت تایلند در اعتر...,Soe Zeya Tun/Reuters,LifeSkills
2026,پنجشنبه ۷ بهمن ۱۴۰۰ - ۱۷:۵۳,ویدیو | صحبت های رئیس فدراسیون فوتبال در ورزشگ...,رئیس فدراسیون فوتبال در بدو ورود به ورزشگاه آز...,به گزارش همشهری آنلاین، شهاب الدین عزیزی خادم ...,Sport
1971,چهارشنبه ۱۵ دی ۱۴۰۰ - ۱۴:۴۳,جدیدترین قیمت گوشت تازه گوسفندی در میادین میوه...,سازمان مدیریت میادین میوه و تره‌بار شهرداری ته...,به گزارش همشهری آنلاین، روابط عمومی سازمان مدی...,City
181,سه‌شنبه ۱۹ بهمن ۱۴۰۰ - ۱۴:۵۶,شکایت بیت امام (ره) از حسین دهباشی,بیت امام راحل از حسین دهباشی مستندساز به دلیل ...,به گزارش همشهری آنلاین به نقل از ایسنا، در پی ...,Politics
2844,چهارشنبه ۲۹ دی ۱۴۰۰ - ۱۵:۳۹,کتاب کودکان و حقوق بشر به زبان انگلیسی منتشر شد,مجموعه ۲ جلدی Childeren and Human Rights/ My s...,به گزارش همشهری آنلاین به نقل از ایلنا، مجموعه...,Culture
324,یکشنبه ۳۰ آبان ۱۴۰۰ - ۰۸:۲۳,جزئیات آتش زدن تابلو دفتر نماینده رباط کریم و ...,نماینده شهرستان‌های رباط کریم و بهارستان گفت: ...,به گزارش همشهری‌آنلاین به نقل از فارس، حسن نور...,Politics
1095,جمعه ۲۹ مرداد ۱۴۰۰ - ۱۳:۲۵,زمان شارژ سوخت خودروهای سنگین کاهش یافت,مدیرکل دفتر حمل کالای سازمان راهداری و حمل و ن...,به گزارش همشهری آنلاین به نقل از صدا و سیما، غ...,Economy
963,جمعه ۱۹ شهریور ۱۴۰۰ - ۱۳:۱۶,فروش فوق العاده ۳ محصول ایران خودرو از پس فردا,گروه صنعتی ایران خودرو بیست و دومین مرحله فروش...,به گزارش همشهری آنلاین به نقل ایران خودرو، در ...,Economy


In [40]:
preprocessor.preprocess("احکام روزه")

['احکام', 'روزه']